# Simulations

In this notebook, we perform simulations. We compare the coefficient before and after the simulation.

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from scipy.stats import pearsonr
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.linear_model import LinearRegression
#import xgboost
from xgboost import XGBRegressor
import joblib
import ptitprince as pt

pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

## Data Import and Processing + Model Import

In [2]:
data = pd.read_csv('data/data.csv')
data['Clim_zone'] = data['Clim_zone'].astype('category')
urban_df = data[['Urban_geoid', 'Urban_name', 'Clim_zone', 'Coastal?']].drop_duplicates().copy()

<ipython-input-2-7c0710de09e2>:1: DtypeWarning: Columns (38) have mixed types. Specify dtype option on import or set low_memory=False.
  data = pd.read_csv('data/data.csv')


In [3]:
model_cuhi_d = joblib.load('models/CUHI_day_summer_xgb_estimator.joblib') 
model_cuhi_n = joblib.load('models/CUHI_night_summer_xgb_estimator.joblib') 
model_suhi_d = joblib.load('models/SUHI_day_summer_xgb_estimator.joblib') 
model_suhi_n = joblib.load('models/SUHI_night_summer_xgb_estimator.joblib') 

In [4]:
features = ['ESABuilt_Area', 'ESAGrass_Area', 'ESATree_Area', 
            'Albedo_Built', 'Albedo_Grass', 'Albedo_Tree', 
            'DEM_urb_CT_act', 'Clim_zone']
labels   = ['CUHI_day_summer', 'CUHI_night_summer', 'SUHI_day_summer', 'SUHI_night_summer']

### Simulation Helper Functions

In [5]:
def get_sim_results(X_test): 
    y_pred_cuhi_d = model_cuhi_d.predict(X_test[features])
    y_pred_cuhi_n = model_cuhi_n.predict(X_test[features])
    y_pred_suhi_d = model_suhi_d.predict(X_test[features])
    y_pred_suhi_n = model_suhi_n.predict(X_test[features])
    X_test['y_pred_cuhi_d'] = y_pred_cuhi_d
    X_test['y_pred_cuhi_n'] = y_pred_cuhi_n
    X_test['y_pred_suhi_d'] = y_pred_suhi_d
    X_test['y_pred_suhi_n'] = y_pred_suhi_n
    return X_test

In [6]:
def apply_regression(df):
    labels = ['y_pred_cuhi_d', 'y_pred_cuhi_n', 'y_pred_suhi_d', 'y_pred_suhi_n']
    X = df['Median Income']
    X = np.array(X).reshape(-1, 1)
    model_cuhi_day = LinearRegression().fit(X, df[labels[0]])
    model_cuhi_night = LinearRegression().fit(X, df[labels[1]])
    model_suhi_day = LinearRegression().fit(X, df[labels[2]])
    model_suhi_night = LinearRegression().fit(X, df[labels[3]])
    return pd.Series({'slope_CUHI_day': model_cuhi_day.coef_[0], 
                      'slope_CUHI_night': model_cuhi_night.coef_[0], 
                      'slope_SUHI_day': model_suhi_day.coef_[0], 
                      'slope_SUHI_night': model_suhi_night.coef_[0]})
    #return pd.Series({'intercept': model.intercept_, 'slope': model.coef_[0]})

In [7]:
def get_coef_results(sim_results):
    coef_result = sim_results.groupby('Urban_geoid').apply(apply_regression)
    coef_result.reset_index(inplace=True)
    coef_result = coef_result.merge(urban_df, how='left', on='Urban_geoid')
    return coef_result

## Baseline

In [8]:
X_base = data.dropna().copy()

In [9]:
sim_results_baseline = get_sim_results(X_base)

In [10]:
coef_result_baseline = get_coef_results(sim_results_baseline)

## Simulation 1: Cool Roofs and Reflective Pavement
To simulate cool roofs and reflective pavements, we set the built albedo to 0.80.

In [11]:
len(data[data['Albedo_Built']>= 0.8])

5

In [12]:
X_albedo = data.dropna().copy()
X_albedo.loc[:, 'Albedo_Built'] = 0.8

In [13]:
sim_results_albedo = get_sim_results(X_albedo)

In [14]:
coef_result_albedo = get_coef_results(sim_results_albedo)

## Simulation 2: Afforestation

We simulate 50% to 100% conversion of grassland to trees. For full conversion of grassland to trees, the grass albedo is replaced by tree albedo as well.

### 50 percent afforestation

In [15]:
X_afforest_50 = data.dropna().copy()
X_afforest_50['ESATree_Area']  = X_afforest_50['ESATree_Area'] + (50/100 * X_afforest_50['ESAGrass_Area']) # add the grass area to the tree area
X_afforest_50['ESAGrass_Area'] = 50/100 * X_afforest_50['ESAGrass_Area'] # divide the grass area by percent afforested area
X_afforest_50['Albedo_Grass']  = X_afforest_50['Albedo_Grass'] * X_afforest_50['ESAGrass_Area'] + X_afforest_50['Albedo_Tree'] * X_afforest_50['ESATree_Area']

In [16]:
sim_results_afforest_50 = get_sim_results(X_afforest_50)

In [17]:
coef_result_afforest_50 = get_coef_results(sim_results_afforest_50)

### 100 percent afforestation

In [18]:
X_afforest_100 = data.dropna().copy()
X_afforest_100['ESATree_Area']  = X_afforest_100['ESATree_Area'] + X_afforest_100['ESAGrass_Area'] # add the grass area to the tree area
X_afforest_100['ESAGrass_Area'] = 0 # Subtract 100 percent of grass area from grass area
X_afforest_100['Albedo_Grass']  = X_afforest_100['Albedo_Tree'] # The albedo of the original grass part is now that of trees

In [19]:
sim_results_afforest_100 = get_sim_results(X_afforest_100)

In [20]:
coef_result_afforest_100 = get_coef_results(sim_results_afforest_100)

## Simulation 3: Afforestation + Albedo Management

### Cool Roofs/Reflective Pavement + 50 percent Afforestation

In [21]:
X_albedo_afforest_50 = data.dropna().copy()
X_albedo_afforest_50['ESATree_Area']  = X_albedo_afforest_50['ESATree_Area'] + (50/100 * X_albedo_afforest_50['ESAGrass_Area']) # add the grass area to the tree area
X_albedo_afforest_50['ESAGrass_Area'] = 50/100 * X_albedo_afforest_50['ESAGrass_Area'] # divide the grass area by percent afforested area
X_albedo_afforest_50['Albedo_Grass']  = X_albedo_afforest_50['Albedo_Grass'] * X_albedo_afforest_50['ESAGrass_Area'] + X_albedo_afforest_50['Albedo_Tree'] * X_albedo_afforest_50['ESATree_Area']
X_albedo_afforest_50.loc[:, 'Albedo_Built'] = 0.8

In [22]:
sim_results_albedo_afforest_50 = get_sim_results(X_albedo_afforest_50)

In [23]:
coef_result_albedo_afforest_50 = get_coef_results(sim_results_albedo_afforest_50)

### Cool Roofs/Reflective Pavement + 100 percent Afforestation

In [24]:
X_albedo_afforest_100 = data.dropna().copy()
X_albedo_afforest_100['ESATree_Area']  = X_albedo_afforest_100['ESATree_Area'] + X_albedo_afforest_100['ESAGrass_Area'] # add the grass area to the tree area
X_albedo_afforest_100['ESAGrass_Area'] = 0 # Subtract 100 percent of grass area from grass area
X_albedo_afforest_100['Albedo_Grass']  = X_albedo_afforest_100['Albedo_Tree'] # The albedo of the original grass part is now that of trees
X_albedo_afforest_100.loc[:, 'Albedo_Built'] = 0.8

In [25]:
sim_results_albedo_afforest_100 = get_sim_results(X_albedo_afforest_100)

In [26]:
coef_result_albedo_afforest_100 = get_coef_results(sim_results_albedo_afforest_100)

## Results

### Median

In [27]:
slopes = ['slope_CUHI_day', 'slope_CUHI_night', 'slope_SUHI_day', 'slope_SUHI_night']

In [28]:
med_results = pd.DataFrame({
    'Baseline': coef_result_baseline[slopes].median(), 
    'Albedo Management': coef_result_albedo[slopes].median(), 
    '50% Afforestation':coef_result_afforest_50[slopes].median(), 
    '100% Afforestation': coef_result_afforest_100[slopes].median(), 
    'Albedo Management + 50% Afforestation': coef_result_albedo_afforest_50[slopes].median(), 
    'Albedo Management + 100% Afforestation': coef_result_albedo_afforest_100[slopes].median()
})

In [29]:
med_results.T

,slope_CUHI_day,slope_CUHI_night,slope_SUHI_day,slope_SUHI_night
Baseline,-0.000010,-9.857110e-06,-0.000078,-0.000023
Albedo Management,-0.000006,-7.914951e-06,-0.000070,-0.000018
50% Afforestation,-0.000010,-7.862130e-06,-0.000066,-0.000012
100% Afforestation,-0.000003,-9.352773e-08,-0.000058,-0.000005
Albedo Management + 50% Afforestation,-0.000008,-9.913711e-06,-0.000065,-0.000012
Albedo Management + 100% Afforestation,0.000001,-2.186349e-06,-0.000054,-0.000010


In [30]:
coef_result_baseline.groupby(['Clim_zone'])[slopes].median()

,slope_CUHI_day,slope_CUHI_night,slope_SUHI_day,slope_SUHI_night
Clim_zone,,,,
Arid,-1.189701e-05,-0.000010,-0.000032,-0.000012
Snow,-1.369508e-05,-0.000015,-0.000114,-0.000035
Temperate,-7.924636e-06,-0.000008,-0.000070,-0.000017
Tropical,-4.779038e-07,0.000004,0.000035,0.000012


In [31]:
s = 'slope_CUHI_day'
med_results = pd.DataFrame({
    'Baseline': coef_result_baseline.groupby(['Clim_zone'])[s].median(), 
    'Albedo Management': coef_result_albedo.groupby(['Clim_zone'])[s].median(), 
    '50% Afforestation':coef_result_afforest_50.groupby(['Clim_zone'])[s].median(), 
    '100% Afforestation': coef_result_afforest_100.groupby(['Clim_zone'])[s].median(), 
    'Albedo Management + 50% Afforestation': coef_result_albedo_afforest_50.groupby(['Clim_zone'])[s].median(), 
    'Albedo Management + 100% Afforestation': coef_result_albedo_afforest_100.groupby(['Clim_zone'])[s].median()
})
med_results.T

Clim_zone,Arid,Snow,Temperate,Tropical
Baseline,-0.000012,-0.000014,-0.000008,-4.779038e-07
Albedo Management,-0.000005,-0.000007,-0.000005,1.641263e-07
50% Afforestation,-0.000004,-0.000015,-0.000008,8.053478e-06
100% Afforestation,-0.000008,-0.000007,-0.000001,8.750326e-06
Albedo Management + 50% Afforestation,-0.000004,-0.000012,-0.000007,6.804287e-06
Albedo Management + 100% Afforestation,-0.000008,0.000005,0.000001,8.660199e-06


In [32]:
s = 'slope_CUHI_night'
med_results = pd.DataFrame({
    'Baseline': coef_result_baseline.groupby(['Clim_zone'])[s].median(), 
    'Albedo Management': coef_result_albedo.groupby(['Clim_zone'])[s].median(), 
    '50% Afforestation':coef_result_afforest_50.groupby(['Clim_zone'])[s].median(), 
    '100% Afforestation': coef_result_afforest_100.groupby(['Clim_zone'])[s].median(), 
    'Albedo Management + 50% Afforestation': coef_result_albedo_afforest_50.groupby(['Clim_zone'])[s].median(), 
    'Albedo Management + 100% Afforestation': coef_result_albedo_afforest_100.groupby(['Clim_zone'])[s].median()
})
med_results.T

Clim_zone,Arid,Snow,Temperate,Tropical
Baseline,-0.000010,-1.504815e-05,-7.642399e-06,3.515970e-06
Albedo Management,-0.000008,-1.306053e-05,-6.823470e-06,3.097254e-06
50% Afforestation,-0.000007,-1.176240e-05,-6.772424e-06,1.103369e-06
100% Afforestation,-0.000007,4.961108e-06,-5.042951e-07,-9.352773e-08
Albedo Management + 50% Afforestation,-0.000008,-1.440389e-05,-8.057840e-06,1.051586e-06
Albedo Management + 100% Afforestation,-0.000006,-6.611091e-07,-3.204070e-06,-1.291323e-06


### Descriptive Table Helper Function

In [33]:
def set_des_results(slope):
    des_results = pd.DataFrame({
        'Baseline': coef_result_baseline[slope].describe(), 
        'Albedo Management': coef_result_albedo[slope].describe(), 
        '50% Afforestation':coef_result_afforest_50[slope].describe(), 
        '100% Afforestation': coef_result_afforest_100[slope].describe(), 
        'Albedo Management + 50% Afforestation': coef_result_albedo_afforest_50[slope].describe(), 
        'Albedo Management + 100% Afforestation': coef_result_albedo_afforest_100[slope].describe()
    })
    return des_results.T

### CUHI Day

In [34]:
set_des_results('slope_CUHI_day')

,count,mean,std,min,25%,50%,75%,max
Baseline,493.0,-1.373286e-05,0.000018,-0.000146,-0.000018,-0.000010,-0.000005,0.000073
Albedo Management,493.0,-7.957183e-06,0.000012,-0.000087,-0.000010,-0.000006,-0.000003,0.000055
50% Afforestation,493.0,-1.408851e-05,0.000021,-0.000122,-0.000019,-0.000010,-0.000004,0.000071
100% Afforestation,493.0,-3.836406e-06,0.000031,-0.000148,-0.000017,-0.000003,0.000011,0.000193
Albedo Management + 50% Afforestation,493.0,-1.111326e-05,0.000018,-0.000118,-0.000015,-0.000008,-0.000003,0.000073
Albedo Management + 100% Afforestation,493.0,9.618521e-07,0.000029,-0.000151,-0.000011,0.000001,0.000014,0.000174


### CUHI Night

In [35]:
set_des_results('slope_CUHI_night')

,count,mean,std,min,25%,50%,75%,max
Baseline,493.0,-0.000012,0.000015,-0.000092,-0.000019,-9.857110e-06,-0.000004,0.000103
Albedo Management,493.0,-0.000010,0.000012,-0.000061,-0.000015,-7.914951e-06,-0.000004,0.000045
50% Afforestation,493.0,-0.000010,0.000012,-0.000066,-0.000015,-7.862130e-06,-0.000003,0.000064
100% Afforestation,493.0,0.000001,0.000017,-0.000067,-0.000006,-9.352773e-08,0.000009,0.000113
Albedo Management + 50% Afforestation,493.0,-0.000012,0.000014,-0.000070,-0.000018,-9.913711e-06,-0.000004,0.000035
Albedo Management + 100% Afforestation,493.0,-0.000003,0.000017,-0.000097,-0.000009,-2.186349e-06,0.000004,0.000048


### SUHI Night

In [36]:
set_des_results('slope_SUHI_day')

,count,mean,std,min,25%,50%,75%,max
Baseline,493.0,-0.000087,0.000077,-0.000388,-0.000130,-0.000078,-0.000039,0.000196
Albedo Management,493.0,-0.000075,0.000061,-0.000301,-0.000108,-0.000070,-0.000037,0.000165
50% Afforestation,493.0,-0.000070,0.000068,-0.000377,-0.000102,-0.000066,-0.000030,0.000194
100% Afforestation,493.0,-0.000066,0.000064,-0.000357,-0.000095,-0.000058,-0.000024,0.000217
Albedo Management + 50% Afforestation,493.0,-0.000066,0.000061,-0.000316,-0.000097,-0.000065,-0.000030,0.000203
Albedo Management + 100% Afforestation,493.0,-0.000060,0.000060,-0.000394,-0.000090,-0.000054,-0.000023,0.000218


### SUHI Night

In [37]:
set_des_results('slope_SUHI_night')

,count,mean,std,min,25%,50%,75%,max
Baseline,493.0,-0.000024,0.000028,-0.000128,-0.000038,-0.000023,-0.000007,0.000162
Albedo Management,493.0,-0.000021,0.000023,-0.000125,-0.000034,-0.000018,-0.000008,0.000107
50% Afforestation,493.0,-0.000014,0.000019,-0.000098,-0.000024,-0.000012,-0.000004,0.000053
100% Afforestation,493.0,-0.000006,0.000016,-0.000066,-0.000013,-0.000005,0.000002,0.000084
Albedo Management + 50% Afforestation,493.0,-0.000014,0.000019,-0.000092,-0.000023,-0.000012,-0.000004,0.000055
Albedo Management + 100% Afforestation,493.0,-0.000012,0.000017,-0.000097,-0.000021,-0.000010,-0.000002,0.000080
